# **一、量子计算云平台量子编程教学**

-----------------------

# 教学大纲
## 1. [基于QCIS的首个量子程序提交](#jump_1)
### &nbsp;&nbsp;&nbsp;&nbsp; 1.1 [准备量子计算云平台登录信息](#jump_1_1)
### &nbsp;&nbsp;&nbsp;&nbsp; 1.2 [量子程序编写](#jump_1_2)
### &nbsp;&nbsp;&nbsp;&nbsp; 1.3 [将程序提交到量子计算云平台](#jump_1_3)
### &nbsp;&nbsp;&nbsp;&nbsp; 1.4 [读取实验结果](#jump_1_4)


# 1 基于QCIS的首个量子程序提交<a id="jump_1"></a>

## 1.1 准备量子计算云平台登录信息<a id="jump_1_1"></a>

In [1]:
from ezQpy import * 
from math import pi
# account = Account(login_key='9d60894fd6692bd00ee529e6bddd8d31', machine_name='应答机A')
account = Account(login_key="9d60894fd6692bd00ee529e6bddd8d31", machine_name="ClosedBetaQC")
#设置用户SDK密钥，选择量子计算机，并创建实例，运行成功后，提示登录成功。

lab_id = "c9329ae3fb6b4862aa5f62edbdf98c98"

Exception: 设置机器名失败，请求接口失败

## 1.2 量子程序编写<a id="jump_1_2"></a>
量子程序有多种书写方式，也有多种运行方式，作为基础教程，本案直接提供一个示例线路，不介绍细节。  
高阶用例请参考后继教程。

In [ ]:
#手动书写第一个量子程序：Bell态制备
#强烈建议设计线路时，直接考虑比特拓扑结构的影响。而不用程序做比特映射操作。
qcis_circuit = '''
H Q7
X Q1
H Q1
CZ Q7 Q1
H Q1
M Q7
M Q1
'''
#可以通过多种方法自行产生待提交的程序。
#以上指令意义请自行补课
#参考QCIS指令集：https://quantumcomputer.ac.cn/Knowledge/detail/all/e3948e8e0fab45c5adcfc730d0a1a3ba.html

## 1.3 将程序提交到量子计算云平台<a id="jump_1_3"></a>

作为入门教程，可以只通过最简单的submit_job()参数来提交一个实验，更多参数见进阶教程。  
函数定义：  
**query_id=$\color{red} {submit\_job}$ (circuit=None, exp_name="exp0",
            parameters=None, values=None, num_shots=12000,
            lab_id=None, exp_id=None, version="version01")**
##### circuit, 量子线路，对于新实验必须提供。  
##### exp_name, 实验集合名称，对于未提供lab_id时，将根据exp_name创建实验集合, 但submit_job函数不返回新建的实验集合ID, 仅建议在快速运行实验时使用。对于同时提供lab_id和exp_name时，exp_name将被忽略。  
##### parameters，values，线路中变量的替换，用于混合编程，见后继教程讲解。  
##### num_shots，实验线路的运行次数，量子实验的特点，拿到的结果是多次实验的统计结果。12比特机型，目前只支持3000次的整数倍。对于其他机型，次数最小为1，最大受用户权限限制。
##### lab_id, 实验集合id，相当于实验目录。
##### exp_id, 实验版本id，不提供线路(circuit)时，可以通过提交exp_id，运行此前保存过的线路。
##### version，实验线路名称。
##### query_id, 返回值，字符类型。用于表征实验运行的id，用于查询实验结果。如果为0/空，则说明实验提交出现异常。
submit_job是由多个基础函数组合而成，参数定义及更丰富的使用形式请参见高阶教程。

In [11]:
query_id = account.submit_job(qcis_circuit)
#对于最简的实验提交，只需要提供实验线路即可。
#但如果想设计自动化程序，各个参数尽量明确复制，并且version等参数在同一个集合内不能重名，也要提前用时间或计数等形式提前产生好。
#因submit_job函数参数较多，建议采用指定参数传参的形式。
#submit_job可以有更多设置，还请关注我们的教程更新。
print(query_id)

保存实验失败, 量子线路指令输入错误，请检查你的输入
0


## 1.4 读取实验结果<a id="jump_1_4"></a>
前面步骤已经将准备好的实验提交到量子计算云平台的量子计算机上并执行，只需通过query_id回读实验结果即可。  
通过submit_job() 将线路传到云平台上的超导量子计算机实体机时，获得实验结果查询id(query_id)，用以查询实验进度，请妥善保存好。  
如果返回query_id为0，则说明报错，报错内容一般会直接在执行过程中输出。  
所以可以考虑用try函数来运行submit_job，对于意外状况，可以直接重新提交失败的实验。以保证全自动程序的运行。
当query_id不为0时，利用query_experiment()可以进行下一步查询工作。

In [ ]:
if query_id:
    result=account.query_experiment(query_id, max_wait_time=360000)
    #result目前为线路返回的数据，各式在内测期间有可能调整，如果已开发程序后期运行出错，可以考虑符合一下这里的格式，并根据具体情况调整。
    #现阶段2023年4月14日，首批内测时，所约定的格式如下：
    #返回值为字典形式，
    #key-"result"为线路执行的原始数据，共计1+num_shots个数据，第一个数据为测量的比特编号和顺序，其余为每shot对应的结果。
    #key-"probability"为线路测量结果的概率统计，经过实时的读取修正后的统计结果。已知number of shots较少时，读取修正后有可能得到部分概率为负值。
    #"probability"中概率为0的结果不回传。
    #当测量比特大于10个时，"probability"为空，请用户自行根据原始数据，配合当时量子计算机的读出保真度自行做修正。相关修正函数在高阶教程中有示例。用户也可以自己完善修正函数。
    #最大等待时间单位为秒，不传递时默认为30秒。因量子程序的执行会有排队的情况，而量子计算机本身有自动校准的时间，如果想跑全自动的程序，等待时间最好大于两者。
    
    #以下是实验结果的显示、使用与保存。
    #打印，显示结果
    print(result)
    #实验结果为原始数据，数据较长。这里不打印，如有兴趣观察实验结果结构，可以选择打印。
    #每次shot的比特测量结果数据，便于灵活使用，如果需要统计结果，可见高阶教程。
    #选出、处理部分结果示例
    value = result
    #print(value)
    #实验结果为原始数据，数据较长。这里不打印，如有兴趣观察实验结果结构，可以选择打印。
    #保存结果
    f = open("./results.txt",'w')
    f.write(str(value))
    f.close()
    print("实验结果已存盘。")
else :
    #实验未运行成功，需要后继重新提交等处理
    print("实验运行异常，需要重新提交或运行")

如果需要保存此时量子计算的参数作为后期参考，如读取修正等，可以运行以下指令：

In [ ]:
res=account.download_config()
#机器完整参数将以json文件形式存储在当前目录。
print(res)

以上即完成基于QCIS的最简实验提交流程。如果需要对实验进行适当的归集，或者半自动、全自动的提交实验，重做指定实验等，可以参考后继的高级篇。  